# PaddlePaddle BYOS

## Pre-requisites

This notebook shows how to use the SageMaker Python SDK to run your code in a local container before deploying to SageMaker's managed training or hosting environments.  This can speed up iterative testing and debugging while using the same familiar Python SDK interface.  Just change your estimator's `train_instance_type` to `local` (or `local_gpu` if you're using an ml.p2 or ml.p3 notebook instance).

In order to use this feature you'll need to install docker-compose (and nvidia-docker if training with a GPU).

**Note, you can only run a single local notebook at one time.**

In [1]:
# !/bin/bash ./utils/setup.sh

## Overview

The **SageMaker Python SDK** helps you deploy your models for training and hosting in optimized, productions ready containers in SageMaker. The SageMaker Python SDK is easy to use, modular, extensible and compatible with TensorFlow, MXNet, PyTorch and Chainer. This tutorial focuses on how to create a convolutional neural network model to train the [Cifar10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) using **PyTorch in local mode**.

### Set up the environment

This notebook was created and tested on a single ml.p2.xlarge notebook instance.

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the sagemaker.get_execution_role() with appropriate full IAM role arn string(s).

In [11]:
import os
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-PaddleNLP'

role = sagemaker.get_execution_role()

In [3]:
# import subprocess

# instance_type = 'local'

# if subprocess.call('nvidia-smi') == 0:
#     ## Set type to GPU if one is present
#     instance_type = 'local_gpu'
    
# print("Instance type = " + instance_type)

### Upload the data
We use the ```sagemaker.Session.upload_data``` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use this later when we start the training job.

In [2]:
WORK_DIRECTORY = './shulexv7/'

data_location = sagemaker_session.upload_data(WORK_DIRECTORY, key_prefix=prefix)
data_location

's3://sagemaker-us-west-2-064542430558/sagemaker/DEMO-PaddleNLP'

## Script Functions

SageMaker invokes the main function defined within your training script for training. When deploying your trained model to an endpoint, the model_fn() is called to determine how to load your trained model. The model_fn() along with a few other functions list below are called to enable predictions on SageMaker.

### [Predicting Functions](https://github.com/aws/sagemaker-pytorch-containers/blob/master/src/sagemaker_pytorch_container/serving.py)
* model_fn(model_dir) - loads your model.
* input_fn(serialized_input_data, content_type) - deserializes predictions to predict_fn.
* output_fn(prediction_output, accept) - serializes predictions from predict_fn.
* predict_fn(input_data, model) - calls a model on data deserialized in input_fn.

The model_fn() is the only function that doesn't have a default implementation and is required by the user for using PyTorch on SageMaker. 

## Create a training job using the sagemaker.PyTorch estimator

The `PyTorch` class allows us to run our training function on SageMaker. We need to configure it with our training script, an IAM role, the number of training instances, and the training instance type. For local training with GPU, we could set this to "local_gpu".  In this case, `instance_type` was set above based on your whether you're running a GPU instance.

After we've constructed our `PyTorch` object, we fit it using the data we uploaded to S3. Even though we're in local mode, using S3 as our data source makes sense because it maintains consistency with how SageMaker's distributed, managed training ingests data.


## SageMaker Training using GPU instance

In [3]:
inputs = {'training': data_location}

print(inputs)

{'training': 's3://sagemaker-us-west-2-064542430558/sagemaker/DEMO-PaddleNLP'}


# prepare pretrained-model

In [4]:
!pip install -r requirements.txt

Looking in indexes: https://opentuna.cn/pypi/web/simple/
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.


In [5]:
from paddlenlp import Taskflow
schema = ['时间', '选手', '赛事名称']
ie = Taskflow("information_extraction", model='uie-base-en', schema=schema, home_path='./uie-base-en')

[2022-10-21 03:56:30,132] [    INFO] - Downloading model_state.pdparams from https://bj.bcebos.com/paddlenlp/taskflow/information_extraction/uie_base_en_v1.1/model_state.pdparams
100%|██████████| 418M/418M [00:50<00:00, 8.66MB/s]   
[2022-10-21 03:57:23,188] [    INFO] - Downloading model_config.json from https://bj.bcebos.com/paddlenlp/taskflow/information_extraction/uie_base_en/model_config.json
100%|██████████| 347/347 [00:00<00:00, 319kB/s]
[2022-10-21 03:57:24,374] [    INFO] - Downloading vocab.txt from https://bj.bcebos.com/paddlenlp/taskflow/information_extraction/uie_base_en/vocab.txt
100%|██████████| 226k/226k [00:03<00:00, 61.6kB/s] 
[2022-10-21 03:57:29,220] [    INFO] - Downloading special_tokens_map.json from https://bj.bcebos.com/paddlenlp/taskflow/information_extraction/uie_base_en/special_tokens_map.json
100%|██████████| 112/112 [00:00<00:00, 104kB/s]
[2022-10-21 03:57:30,243] [    INFO] - Downloading tokenizer_config.json from https://bj.bcebos.com/paddlenlp/taskflow/

In [6]:
#upload uie-base-en pretrain

uie_en_model_s3 = sagemaker.Session().upload_data(path = "./uie-base-en/taskflow/information_extraction/uie-base-en", key_prefix="model_uie_base_en")

In [9]:
from sagemaker.pytorch import PyTorch

hyperparameters = {'train_path': '/opt/ml/input/data/training/train.txt', 
                   'dev_path': '/opt/ml/input/data/training/dev.txt', 
                   'save_dir': '/opt/ml/model', 
                   'learning_rate': 1e-5, 
                   'batch_size': 16, 
                   'max_seq_len':512, 
                   'num_epochs': 10, 
                   'model': 'uie-base',
                   'seed': 1000,
                   'logging_steps': 1,
                   'valid_steps': 10,
                   'device': 'gpu'}

instance_type = 'ml.p3.2xlarge'  # 'ml.p3.2xlarge' or 'ml.p3.8xlarge' or ...

#git_config = {'repo': 'https://github.com/whn09/paddlenlp_sagemaker.git', 'branch': 'main'}

estimator = PyTorch(entry_point='finetune.py',
                    source_dir='./',
                           # git_config=git_config,
                    role=role,
                    hyperparameters=hyperparameters,
                    framework_version='1.9.1',
                    py_version='py38',
                    script_mode=True,
                    instance_count=1,  # 1 or 2 or ...
                    instance_type=instance_type,
                    # Parameters required to enable checkpointing
                    checkpoint_s3_uri=uie_en_model_s3, #使用你自己用来保存/加载模型的s3桶地址, 注意桶需要在us-east-1
                    checkpoint_local_path="/opt/ml/checkpoints")

estimator.fit(inputs)

2022-10-21 05:26:26 Starting - Starting the training job...
2022-10-21 05:26:54 Starting - Preparing the instances for trainingProfilerReport-1666329986: InProgress
.........
2022-10-21 05:28:21 Downloading - Downloading input data...
2022-10-21 05:28:56 Training - Downloading the training image.....................
2022-10-21 05:32:23 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-10-21 05:32:26,084 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-10-21 05:32:26,112 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-10-21 05:32:26,119 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-10-21 05:32:40,776 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python -m pip install -r requir

In [12]:
training_job_name = estimator.latest_training_job.name
# training_job_name = 'xxx'
print(training_job_name)

pytorch-training-2022-10-21-05-25-33-950


# Deploy the trained model to prepare for predictions

The deploy() method creates an endpoint (in this case locally) which serves prediction requests in real-time.

In [ ]:
!rm -rf model.tar.gz
!rm -rf model_*
!rm -rf inference.*
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz .
!tar -xvf model.tar.gz

In [34]:
!rm -rf model-inference-gpu.tar.gz
# !cp inference.* model/
!cp ./model_best/* model/
# !cp model/code/requirements_gpu.txt model/code/requirements.txt
!cd model && tar -czvf ../model-inference-gpu.tar.gz *
!aws s3 cp model-inference-gpu.tar.gz s3://$bucket/$training_job_name/output/model-inference-gpu.tar.gz

code/
code/infer.py
code/.ipynb_checkpoints/
code/.ipynb_checkpoints/infer_gpu-checkpoint.py
code/.ipynb_checkpoints/model-checkpoint.py
code/.ipynb_checkpoints/uie_predictor-checkpoint.py
code/uie_predictor.py
code/infer_cpu.py
code/requirements.txt
code/requirements_gpu.txt
code/model.py
code/infer_gpu.py
code/requirements_cpu.txt
inference.pdiparams
inference.pdiparams.info
inference.pdmodel
model_config.json
model_state.pdparams
special_tokens_map.json
tokenizer_config.json
vocab.txt
upload: ./model-inference-gpu.tar.gz to s3://sagemaker-us-west-2-064542430558/pytorch-training-2022-10-21-05-25-33-950/output/model-inference-gpu.tar.gz


In [35]:
# instance_type = 'local'
# instance_type = 'ml.m5.xlarge'
instance_type = 'ml.g4dn.xlarge'

# predictor = estimator.deploy(initial_instance_count=1, instance_type=instance_type)

from sagemaker.pytorch.model import PyTorchModel

pytorch_model = PyTorchModel(model_data='s3://{}/{}/output/model-inference-gpu.tar.gz'.format(bucket, training_job_name), role=role,
                             entry_point='infer_gpu.py', framework_version='1.9.0', py_version='py38', model_server_workers=4)  # TODO [For GPU], model_server_workers=6

predictor = pytorch_model.deploy(instance_type=instance_type, initial_instance_count=1)

---------------!

# Invoking the endpoint

In [36]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()

In [37]:
texts = ['She had long silky-looking white hair']

import time
start = time.time()
outputs = predictor.predict(texts)
end = time.time()
print('outputs: ', outputs)
print('time:', end-start)

outputs:  [{'person': [{'text': 'She', 'start': 0, 'end': 3, 'probability': 0.5287898182868958}], 'occupation': [{'text': 'She', 'start': 0, 'end': 3, 'probability': 0.5269210934638977}], 'abuse': [{'text': 'hair', 'start': 33, 'end': 37, 'probability': 0.32205232977867126}, {'text': 'She', 'start': 0, 'end': 3, 'probability': 0.6202351450920105}], 'status': [{'text': 'She', 'start': 0, 'end': 3, 'probability': 0.5892147421836853}], 'location': [{'text': 'She', 'start': 0, 'end': 3, 'probability': 0.4177761673927307}, {'text': 'hair', 'start': 33, 'end': 37, 'probability': 0.7394289970397949}], 'age': [{'text': 'She', 'start': 0, 'end': 3, 'probability': 0.31551605463027954}], 'family': [{'text': 'She', 'start': 0, 'end': 3, 'probability': 0.38014349341392517}], 'parts of body': [{'text': 'hair', 'start': 33, 'end': 37, 'probability': 0.6396911144256592}], 'color': [{'text': 'white', 'start': 27, 'end': 32, 'probability': 0.5680398941040039}], 'gender': [{'text': 'She', 'start': 0, 'en

# Clean-up

Deleting the local endpoint when you're finished is important since you can only run one local endpoint at a time.

In [33]:
# estimator.delete_endpoint()
predictor.delete_endpoint()